# Environment

In [1]:
# !pip install langchain langchain_openai rich
# !pip install pypdf chromadb rapidocr-onnxruntime

# Import

In [2]:
# 匯入套件和金鑰
import os
from rich import print as pprint
from langchain_openai import ChatOpenAI

from dotenv import load_dotenv
load_dotenv()

from langchain_community.document_loaders import PyPDFLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

In [3]:
# load LLM model
chat_model = ChatOpenAI(model_name="gpt-3.5-turbo", api_key=os.getenv("OPENAI_API_KEY"))

In [4]:
# load PDF file
pdf_filepath = '/Users/weikai/Library/CloudStorage/Dropbox/paper/CCWu'
pdf_filename = 'CCWu 2021 Modified distributed Bragg reflector for protecting organic light-emitting diode displays against ultraviolet light.pdf'

loader = PyPDFLoader(file_path=os.path.join(pdf_filepath, pdf_filename))
docs = loader.load()
pprint(f'The PDF file has {len(docs)} pages')

# pprint(docs[0])

The PDF file has 12 pages

In [5]:
# load embeddings model
embeddings_model=OpenAIEmbeddings(model='text-embedding-3-large')
index = VectorstoreIndexCreator(embedding=embeddings_model).from_loaders([loader])

/Users/weikai/opt/anaconda3/envs/OLED_RAG/lib/python3.12/site-packages/langchain/indexes/vectorstore.py:129: UserWarning: Using InMemoryVectorStore as the default vectorstore.This memory store won't persist data. You should explicitlyspecify a vectorstore when using VectorstoreIndexCreator
  warnings.warn(


In [6]:
query = "What is the key idea of this paper?"
response = index.query(llm=chat_model, question=query)
pprint(response)

The key idea of this paper is to develop a UV/HEV-blocking structure that can provide effective UV/HEV blocking 
(absorption), high visible transmission, and a sharp transmission transition at specific wavelength ranges while 
also being simple, cost-effective, and capable of maintaining the emission characteristics of displays like OLEDs. 
The paper discusses the challenges of achieving these optical properties and explores the use of optical 
structures, such as DBR (Distributed Bragg Reflector) pairs, in combination with UV absorption materials to meet 
these requirements. It also emphasizes the importance of protecting OLED panels against solar degradation by using 
a combination of polarizers and UV-blocking DBRs.

In [7]:
Chroma.from_documents(documents=docs,
                      embedding=embeddings_model,
                      persist_directory='./chromadb',
                      collection_metadata={"hnsw:space": "cosine"})

In [8]:
db = Chroma(persist_directory='./chromadb', 
            embedding_function=embeddings_model)

In [18]:
retriever = db.as_retriever(search_type="similarity",
                            search_kwargs={"k": 2})

retrieved_docs = retriever.invoke("What is the key idea of this paper?")
print(f'傳回 {len(retrieved_docs)} 筆資料')

pprint( retrieved_docs )

傳回 2 筆資料


[
    Document(
        page_content='Research Article Vol. 29, No. 5 / 1 March 2021 / Optics Express 7660\nis the peak wavelength 
of the first ripple in the conventional DBR (Fig. 4(a)), the peak-to-valley\nmodulation of the total electric field
in the conventional DBR (Fig. 5(a)) is larger than that in the\n2ndmodified DBR (Fig. 5(b)). Since the total field 
is composed of waves propagating toward 
+z\nand-z,largerpeak-to-valleymodulationindicatesthatmoresimilarintensitiesbetween 
+zand-z\npropagatingwaves(strongercollectivereflection). 
Meanwhile,thelowpeak-to-valleymodulation\ninthe2ndmodifiedDBRshowsthatadjustingthelayerthicknessesinthefirstpairand
thelast\npair can effectively suppressing the intensity of the -z propagating (reflecting) wave and 
thus\nsubstantially enhance the overall transmittance.\nFig. 5.Simulated distributions of total electric field 
intensities in the 6-pair DBRs for\nwavelengthsof300nm,370nm,and493nm: 
(a)theconventionalDBRstructure,(b)2nd\nmodified DBR 
design.\nFollowingthe2ndmodifiedDBRdesignstrategy,Fig.6showscalculatedT-R-Aspectrafor\n6-pair 2ndmodified DBR 
structures with varied λTof 350nm to 390nm (while more detailed\nproperties are summarized in Table S2 of 
Supplement 1). It is seen that boundaries/edges of\ntheir transmittance and reflection spectra can be readily 
blue-shifted with reducing the target\nresonant wavelength λT(simply by adjusting the layer thicknesses), if 
desired (e.g., when deeper\nblue OLED pixels is desired and adopted and thus a different balance between blue EL 
and UV\nblockingisrequired). 
This(easyandprecisetuningofthetransitionwavelengthandsharpness\nofthetransmittancespectrum)perhapsisaparticularadva
ntageandconvenienceofusingthe\nDBRstructureforUVblocking,comparedtootherapproaches(e.g.,simplyusingabsorption\nprop
erties of materials). From these results, it is also clear that for a blue OLED pixel with\nELpeakingaround460nm, 
λTof370nmgivesareasonablebalancebetweenblueELandUV\nblocking.\nFig. 6.Calculated (a) transmittance spectra, (b) 
reflectance spectra, and (c) absorptance\nspectra for 2ndmodified DBR structures with 6 pairs and λT=350nm, 360nm, 
370nm,\n380nm, and 390nm.\nIt is also worthy of stressing other advantages of the current modified DBR design 
compared\nto conventional DBR designs. Although DBR is a common technology, it is still generally',
        metadata={
            'page': 6,
            'source': '/Users/weikai/Library/CloudStorage/Dropbox/paper/CCWu/CCWu 2021 Modified distributed Bragg 
reflector for protecting organic light-emitting diode displays against ultraviolet light.pdf'
        }
    ),
    Document(
        page_content='Research Article Vol. 29, No. 5 / 1 March 2021 / Optics Express 7664\n(architecture 
A)sufferlargevoltageshift( 
∆V=2.62Vafter25testcycles),clearlymanifesting\ndetrimentaleffectofsunlightirradiationonOLEDs. 
Thepolarizerinarchitecture B(conventional\nOLEDpanelstructure)providespartialUVprotectionandthusreduces 
∆V(∆V=1.3∼1.5Vafter\n25 cycles). Finally, combining the polarizer and UV-blocking DBR (architecture C) 
provides\nfurther protection of OLED panels against solar degradation, and ∆V after 25 test cycles 
is\nfurtherreduceddownto ∆V∼0.8V,confirmingeffectivenessofthedesignedUV-blockingDBRs\nherein. 
Itisalsonoticedthatboth6-pairand7-pairUV-blockingDBRsstructuresprovidesimilar\nprotection against UV/HEV light. 
Table S4 in Supplement 1 further provides a summary of\nthe luminanceof theOLED device undera sameoperation voltage
(4V) before andafter being\nsubjectedtothesolartestarchitectures A,B,and 
C(with6-pair2ndmodifiedDBRstructure),\nagain confirming effectiveness of the designed UV-blocking DBR 
structures.\n5. Conclusion\nIn summary, we developed modified DBR structures composed of UV-absorbing and 
dielectric\nmaterials for protecting OLED panels against UV/HEV irradiation and degradation, 
thus\nstrengtheningtheiroutdoorapplications. Bytuningthicknessesofhigh-index/low-indexlayers\nin the DBR p

In [10]:
str_parser = StrOutputParser()
template = (
    "請根據以下內容加上自身判斷回答問題:\n"
    "{context}\n"
    "問題: {question}"
    )
prompt = ChatPromptTemplate.from_template(template)
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | chat_model
    | str_parser
)

In [11]:
pprint(chain.invoke("What is the key idea of this paper?"))

這篇論文的關鍵觀點是開發了修改後的分佈布拉格反射器結構，由UV吸收和介電材料組成，用於保護OLED面板免受UV/HEV輻射和降
解，從而增強它們在戶外應用的能力。通過調節DBR對的高折射率/低折射率層的厚度，設計和製造了具有多個顯著優點的6對和7對U
V阻擋結構，例如整個光譜範圍內高達91％的傳遞率幾乎恒定，深藍/近紫外光區域的傳遞率急劇下降，以及有效防護光譜範圍內的U
V/HEV光，使其適用於OLED顯示面板，對顯示顏色和效率等幾乎沒有影響。

In [12]:
rag_chain_from_docs = (
    prompt
    | chat_model
    | StrOutputParser()
)

In [13]:
from langchain_core.runnables import RunnableParallel

rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

In [14]:
def chat(query):
    output = {}
    curr_key = None
    for chunk in rag_chain_with_source.stream(query):
        for key in chunk:
            if key not in output:
                output[key] = chunk[key]
            else:
                output[key] += chunk[key]
            if key != curr_key:
                print(f"\n\n{key}: {chunk[key]}", end="", flush=True)
            else:
                print(chunk[key], end="", flush=True)
            curr_key = key
chat("What is the key idea of this paper?")



question: What is the key idea of this paper?

context: [Document(page_content='Research Article Vol. 29, No. 5 / 1 March 2021 / Optics Express 7660\nis the peak wavelength of the first ripple in the conventional DBR (Fig. 4(a)), the peak-to-valley\nmodulation of the total electric field in the conventional DBR (Fig. 5(a)) is larger than that in the\n2ndmodified DBR (Fig. 5(b)). Since the total field is composed of waves propagating toward +z\nand-z,largerpeak-to-valleymodulationindicatesthatmoresimilarintensitiesbetween +zand-z\npropagatingwaves(strongercollectivereflection). Meanwhile,thelowpeak-to-valleymodulation\ninthe2ndmodifiedDBRshowsthatadjustingthelayerthicknessesinthefirstpairandthelast\npair can effectively suppressing the intensity of the -z propagating (reflecting) wave and thus\nsubstantially enhance the overall transmittance.\nFig. 5.Simulated distributions of total electric field intensities in the 6-pair DBRs for\nwavelengthsof300nm,370nm,and493nm: (a)theconventiona